# Predicting Material Quantity time-series with the DeepAR built-in algorithm on Amazon SageMaker

Basado en la data enviada

## Download and pre-process data set

In [ ]:
#

In [ ]:
minYear  = 2008
maxYear  = 2018
avg_quantity = 0

# Our model will predict quantity for the next 'prediction_length' weeks
prediction_length = 24

In [ ]:
import csv, boto3, json, sagemaker
import numpy as np

In [ ]:

f = open(filename, 'r')
data = csv.reader(f,delimiter=';')

dataset={}
x=[]
y=[]
count=1
prevYear=0

for row in data:
        # Remove empty strings caused by multiple spaces between columns
        row = filter(None, row)
        #print(row)
        try:
            year=row[2]
            
            quantity=float(row[4])+avg_quantity
            # Data for plotting
            # x list=counter, y list=quantity
            x.append(count)
            y.append(float(quantity))
            count=count+1

            # Data for training
            # dictionary: key=year, value=list of ordered daily temperatures
            if (year != prevYear):
                dataset[year]=[]
                prevYear=year
            dataset[year].append(float(quantity))
        except IndexError as error:
            print("Ignoring line count: ", count,error)

In [ ]:
nb_samples_per_year = list(map(lambda x: len(x), (dataset[str(year)] for year in range(minYear, maxYear+1))))
nb_samples_per_year = np.unique(nb_samples_per_year).tolist()
print(nb_samples_per_year)
assert nb_samples_per_year == [52, 53]

## Plot data set

In [ ]:
import matplotlib.pyplot as plt

nbSamples=len(x)
print('Number of samples: %d' % nbSamples)

fig=plt.figure(figsize=(64, 16))
plt.plot(x,y)
plt.show()

## Prepare training set and test set

In [ ]:
# According to https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html
# - remove the last 'prediction_length' points from each time series
# - use the full data set for testing
# - no need to shuffle the training set: dictionaries are not ordered :)

trainingSet = dataset.copy()
trainingSet[year] = { year: dataset[year][:-prediction_length] for year in dataset.keys() }

testSet = dataset.copy()

In [ ]:
trainingSet.pop('2018', None)
testSet.pop('2018', None)


In [ ]:
train_key      = 'deepar_training.json'
test_key       = 'deepar_test.json'

def writeDataset(filename, data): 
    file=open(filename,'w')
    for year in data.keys():
        # One JSON sample per line
        line = "\"start\":\"{}-01-01 00:00:00\",\"target\":{}".format(year,data[year])
        file.write('{'+line+'}\n')

In [ ]:
writeDataset(train_key, trainingSet)        
writeDataset(test_key, testSet)

In [ ]:
!head -20 deepar_training.json

## Upload training set and test set to S3

In [ ]:
bucket         = 'forecasting-data'
prefix         = 'prefix/curated'

train_prefix   = '{}/{}'.format(prefix, 'train')
test_prefix    = '{}/{}'.format(prefix, 'test')
output_prefix  = '{}/{}'.format(prefix, 'output')

In [ ]:
sagemaker_session = sagemaker.Session()
role              = sagemaker.get_execution_role()
region            = boto3.Session().region_name

train_path  = sagemaker_session.upload_data(train_key, bucket=bucket, key_prefix=train_prefix)
test_path   = sagemaker_session.upload_data(test_key,  bucket=bucket, key_prefix=test_prefix)
output_path = 's3://{}/{}'.format(bucket, output_prefix)

print(train_path)
print(test_path)
print(output_path)

## Configure training job

In [ ]:
containers = {
    'us-east-1': '522234722520.dkr.ecr.us-east-1.amazonaws.com/forecasting-deepar:latest',
    'us-east-2': '566113047672.dkr.ecr.us-east-2.amazonaws.com/forecasting-deepar:latest',
    'us-west-2': '156387875391.dkr.ecr.us-west-2.amazonaws.com/forecasting-deepar:latest',
    'eu-west-1': '224300973850.dkr.ecr.eu-west-1.amazonaws.com/forecasting-deepar:latest'
}

image_name = containers[region]

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    #train_instance_type='ml.c4.8xlarge',
    train_instance_type='ml.m4.2xlarge',
    base_job_name='weekly-quantity',
    output_path=output_path
)

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html

hyperparameters = {
    "time_freq": 'W', # weekly series
    "context_length": prediction_length,
    "prediction_length": prediction_length, # number of data points to predict
    "num_cells": "40",
    "num_layers": "2",
    "likelihood": "gaussian",
    "epochs": "250",
    "mini_batch_size": "32",
    "learning_rate": "0.00001",
    "dropout_rate": "0.05",
    "early_stopping_patience": "10" # stop if loss hasn't improved in 10 epochs
}

estimator.set_hyperparameters(**hyperparameters)

## Run training job

In [ ]:
print(train_path)
print(test_path)
print(output_path)

data_channels = {"train": train_path, "test": test_path}

In [ ]:
%%time

estimator.fit(inputs=data_channels)

## Deploy model

In [ ]:
%%time

job_name = estimator.latest_training_job.name

endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.2xlarge',
    deployment_image=image_name,
    role=role
)

predictor = sagemaker.predictor.RealTimePredictor(
    endpoint_name, 
    sagemaker_session=sagemaker_session, 
    content_type="application/json")

## Build JSON-formatted prediction request

In [ ]:
q1 = '0.1'         # compute p10 quantile
q2 = '0.9'         # compute p90 quantile
num_samples = 100  # predict 100 sample series
    
def buildPredictionData(year, data):
    year_quantity = data[str(year)]
    s = {"start": "{}-01-01 00:00:00".format(year), "target": year_quantity}
    series = []
    series.append(s)
    configuration = {
        "output_types": ["mean", "quantiles", "samples"],
        "num_samples": num_samples,
        "quantiles": [q1, q2]
    }
    http_data = {
        "instances": series, 
        "configuration": configuration
    }

    response = json.dumps(http_data)
    

    return response

## Get predicted series from response

In [ ]:
def getPredictedSeries(result):
    import random
    json_result = json.loads(result)
    y_data      = json_result['predictions'][0]
    y_mean      = y_data['mean']
    y_q1        = y_data['quantiles'][q1]
    y_q2        = y_data['quantiles'][q2]
    y_sample    = y_data['samples'][random.randint(0, num_samples)]

    print("Mean: %s\n" % y_mean)
    print("Quartile %s: %s\n" % (q1, y_q1))
    print("Quartile %s: %s\n" % (q2, y_q2))
    return y_mean, y_q1, y_q2, y_sample

## Plot predicted series and ground truth

In [ ]:
def plotSeries(result, truth=False, truth_data=None, truth_label=None):
    x = range(0,prediction_length)
    y_mean, y_q1, y_q2, y_sample = getPredictedSeries(result)
    plt.gcf().clear()
    mean_label,   = plt.plot(x, y_mean, label='mean')
    q1_label,     = plt.plot(x, y_q1, label=q1)
    q2_label,     = plt.plot(x, y_q2, label=q2)
    sample_label, = plt.plot(x, y_sample, label='sample')

    if truth:
        ground_truth, = plt.plot(x, truth_data, label=truth_label)
        plt.legend(handles=[ground_truth, q2_label, mean_label, q1_label, sample_label])
    else:
        plt.legend(handles=[q2_label, mean_label, q1_label, sample_label])
    plt.yticks(np.arange(5.0, 12.0, 0.5))
    plt.show()

## Example: predict Quantity for the last 'prediction_length' days and compare to ground truth

In [ ]:
year = 2016 # “He who controls the past controls the future. He who controls the present controls the past.”

prediction_data = buildPredictionData(year, trainingSet)

result = predictor.predict(prediction_data).encode('utf-8')



## Example: predict Quantity for the next 'prediction_length' days

In [ ]:
# Predict quantity for the next 'prediction_length' weeks of 2018
year = 2018
year_quantity={}
year_quantity[str(year)] = np.random.normal(9, 1.5, 90).tolist()

prediction_data = buildPredictionData(year, year_quantity)

result = predictor.predict(prediction_data).encode('utf-8')
print("Result...")
print(result)
filename      = 'results.csv'
file=open(filename,'w')
file.write(result)


## Clean up

In [ ]:
print(job_name)
sm = boto3.client('sagemaker')
sm.delete_endpoint(EndpointName=job_name)
sm.delete_endpoint_config(EndpointConfigName=job_name)
sm.delete_model(ModelName=job_name)